# Data Transformation

In [16]:
specificColumnsDf = df.select("firstname", "lastname", "gender", "location", "level")
specificColumnsDf.show()

# Further transforming to rename the column level to subscription_type.

In [18]:
renamedColumnsDF = specificColumnsDf.withColumnRenamed("level", "subscription_type")
renamedColumnsDF.show()

# Loading into Azure SQL Data Warehouse

In [20]:

dwTable= "mytable001"
dwDatabase = "sqldwprod"
dwServer = "fastf1prod" 
dwUser = "facilities"
dwPass = "P@ssw0rd____"
dwJdbcPort =  "1433"
dwJdbcExtraOptions = "encrypt=true;trustServerCertificate=true;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
sqlDwUrl = "jdbc:sqlserver://" + dwServer + ".database.windows.net:" + dwJdbcPort + ";database=" + dwDatabase + ";user=" + dwUser+";password=" + dwPass + ";$dwJdbcExtraOptions"
sqlDwUrlSmall = "jdbc:sqlserver://" + dwServer + ".database.windows.net:" + dwJdbcPort + ";database=" + dwDatabase + ";user=" + dwUser+";password=" + dwPass


tempDir = "wasbs://dbdemo01tmp@databrickstore.blob.core.windows.net/tempDirs"

#sc._jsc.hadoopConfiguration().set(
#  "fs.azure.account.key.<your-storage-account-name>.blob.core.windows.net",
#  "<your-storage-account-access-key>")
acntInfo = "fs.azure.account.key.databrickstore.blob.core.windows.net"
sc._jsc.hadoopConfiguration().set(
  acntInfo, 
  "S1PtMWvUw5If1Z8FMzXAxC7OMw9G5Go8BGCXJ81qpFVYpZ9dpXOnU4zlg0PbldKkbLIbmbv02WoJsgYLGKIfgg==")

## Loading transformed dataframe (renamedColumnsDF) into SQLDW
spark.conf.set("spark.sql.parquet.writeLegacyFormat","true")

## This snippet creates a table called 'dwTable' in the SQL database.
#df.write \
#  .format("com.databricks.spark.sqldw") \
#  .option("url", "jdbc:sqlserver://<the-rest-of-the-connection-string>") \
#  .option("forward_spark_azure_storage_credentials", "true") \
#  .option("dbtable", "my_table_in_dw_copy") \
#  .option("tempdir", "wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>") \
#  .save()

renamedColumnsDF.write \
  .format("com.databricks.spark.sqldw") \
  .option("url", sqlDwUrlSmall) \
  .option("dbtable", dwTable) \
  .option( "forward_spark_azure_storage_credentials","true") \
  .option("tempdir", tempDir) \
  .mode("overwrite") \
  .save()